In [1]:
import json
from itertools import cycle
from collections import defaultdict
import uuid
import numpy as np
from vizbuilder import Html, components, require

In [9]:
colors_iter = cycle(["red", "blue", "green", "orange", "purple"])
colors = defaultdict(lambda: next(colors_iter))

rand = np.random.RandomState(42)
data = [
        {"category": k, "subcategory": j, "start": int(i), "duration": rand.choice(20), "color": colors[k]}
        for k in ["A", "B", "C", "D", "E"] for j in range(3) for i in np.cumsum(rand.choice(20, 20) + 20) 
    ]

In [7]:
data = [
        {"category": k, "start": int(i), "duration": rand.choice(20), "color": colors[k]}
        for k in ["A", "B", "C", "D", "E"] for i in np.cumsum(rand.choice(20, 20) + 20) 
    ]

In [12]:
# Original Gantt Chart (without scrubber for comparison)
container_id = f"d3-container-{uuid.uuid4().hex[:8]}"   
user_code = f"""
    const data={json.dumps(data)};
    const container = d3.select("#{container_id}").datum(data);
    const chart = ganttChart().scrubberEnabled(false);  // Disable scrubber for this demo
    container.call(chart);
    window.addEventListener('resize', (() => container.call(chart)));
"""

html = Html.body(
    Html.h3("Basic Gantt Chart (no scrubber)"),
    Html.div(id_=container_id),
    Html.script(require(modules=["d3", "ganttChart"], user_code=user_code, clear_cache=True))
)
display(html)

In [6]:
# Enhanced Gantt Chart with Interactive Scrubber and Linked Data Display
chart_container_id = f"chart-container-{uuid.uuid4().hex[:8]}"
display_container_id = f"display-container-{uuid.uuid4().hex[:8]}"

user_code = f"""
    //console.log("Starting to load modules...");
    
    const data={json.dumps(data)};
    
    try {{
        // Test if modules are available
        console.log("d3 available:", typeof d3);
        console.log("ganttChart available:", typeof ganttChart);
        console.log("dataDisplay available:", typeof dataDisplay);
        
        // Create the linked data display component first
        const displayContainer = d3.select("#{display_container_id}");
        const displayComponent = dataDisplay().title("Timeline Entries at Scrubber Position");
        displayContainer.call(displayComponent);
        //console.log("Data display component created successfully");
        
        // Create the Gantt chart with scrubber enabled
        const chartContainer = d3.select("#{chart_container_id}").datum(data);
        const chart = ganttChart().scrubberEnabled(true);
        
        // Link the components using the event system BEFORE rendering chart
        chart.on("scrubberMove", function(event) {{
            // console.log("Scrubber moved, intersected data:", event.intersectedData);
            // Update the data display with intersected entries
            const displayNode = displayContainer.node();
            displayNode.updateData(event.intersectedData);
        }});
        
        // Now render the chart (this will emit the initial scrubberMove event)
        chartContainer.call(chart);
        //console.log("Gantt chart created successfully");
        
        // Handle window resize for chart
        window.addEventListener('resize', (() => chartContainer.call(chart)));
        
        //console.log("All components initialized successfully");
        
    }} catch (error) {{
        console.error("Error in chart initialization:", error);
        console.error("Error stack:", error.stack);
    }}
"""

html = Html.body(
    Html.h3("Interactive Gantt Chart with Scrubber"),
    Html.p("Drag the orange scrubber line to see which timeline entries intersect. The data display below updates automatically."),
    Html.div(id_=chart_container_id, style="height: 350px; margin-bottom: 20px;"),
    Html.div(id_=display_container_id, style="height: 300px;"),
    Html.script(require(modules=["d3", "ganttChart", "dataDisplay"], user_code=user_code, clear_cache=True))
)
display(html)

## Scrubber Feature Documentation

The enhanced Gantt chart now includes a horizontal scrubber feature with the following capabilities:

### Key Features:
- **Interactive Scrubber**: Orange draggable line that can be positioned anywhere on the timeline
- **Visual Feedback**: Timeline entries that intersect with the scrubber remain at full opacity, while others are dimmed to 30% opacity
- **Event System**: Uses D3's dispatch pattern for loose coupling between components
- **Modular Design**: Linked components communicate through events, not direct coupling

### API Overview:

#### Gantt Chart Configuration:
```javascript
const chart = ganttChart()
    .scrubberEnabled(true)  // Enable/disable scrubber (default: true)
    .width(800)            // Chart width
    .height(400);          // Chart height
```

#### Event Handling:
```javascript
chart.on("scrubberMove", function(event) {
    // event.position: Current scrubber position in data coordinates
    // event.intersectedData: Array of timeline entries intersecting the scrubber
});

chart.on("scrubberStart", function(event) {
    // Fired when scrubber drag starts
});

chart.on("scrubberEnd", function(event) {
    // Fired when scrubber drag ends
});
```

#### Data Display Component:
```javascript
const displayComponent = dataDisplay()
    .title("Custom Title")     // Set component title
    .width(600)               // Component width
    .height(300);             // Component height

// Note: Avoid naming conflicts - don't use 'const dataDisplay = dataDisplay()'
```

### Design Patterns Used:
1. **Observer Pattern**: Components observe scrubber events
2. **Modular Architecture**: Each component is self-contained and reusable  
3. **Event-Driven Communication**: Loose coupling via D3 dispatch system
4. **Progressive Enhancement**: Scrubber can be enabled/disabled without breaking existing functionality

### Common Pitfalls:
- **Variable Naming**: Avoid `const dataDisplay = dataDisplay()` - this creates a naming conflict
- **Component Order**: Ensure all components are loaded before referencing them
- **Event Timing**: Components must be rendered before event handlers are attached